In [133]:
import pandas as pd
import datetime as dt

def determine_season(month):
    if month in [9,10,11]:
        return 'fall'
    elif month in [12,1,2]:
        return 'winter'
    elif month in [3,4,5]:
        return 'spring'
    else:
        # in my opinion, the best season
        return 'summer'

bike_months = []
for i in range(1,10):
    df = pd.read_csv('0' + str(i) + '2020.csv')
    # create a new df to contain the unique dates
    cleaned_df = df['date'].drop_duplicates().to_frame().reset_index().drop(columns=['index'])
    # add a new column using the hour of the start time of a trip
    df['hour'] = df['start_time'].apply(lambda x : int(x[0:2]))

    # creating columns for usertypes customer and suscriber with the total user count
    usertype = df.groupby(by=['date', 'hour'])['usertype'].value_counts().to_frame().unstack(level='usertype')
    usertype.reset_index(inplace=True)
    usertype.columns = ['date', 'hour', 'customer', 'subscriber']

    # create the mean and std columns for the trip duration grouped by date and hour
    # then add it onto the previous dataframe to merge with cleaned_df
    trip_stat = df.groupby(by=['date', 'hour'])['tripduration'].agg(['mean', 'std'])
    trip_stat.reset_index(inplace=True)
    usertype['trip_duration_mean'] = trip_stat['mean'].round(2)
    usertype['trip_duration_std'] = trip_stat['std'].round(2)
    usertype.fillna(0, inplace=True)
    
    # remove trailing zeroes
    usertype['subscriber'] = usertype['subscriber'].apply(lambda x : int(format(x, '.0f')))
    usertype['customer'] = usertype['customer'].apply(lambda x : int(format(x, '.0f')))

    cleaned_df = cleaned_df.merge(usertype, how='inner', left_on='date', right_on='date')
    # month column for the month of the year
    cleaned_df['month'] = cleaned_df['date'].apply(lambda x : int(x[5:7]))
    # season column that is one of winter, fall, spring, and summer
    cleaned_df['season'] = cleaned_df['month'].apply(lambda x : determine_season(x))
    # a weekday column for the day of the week: monday-sunday (0-6)
    cleaned_df['weekday'] = cleaned_df['date'].apply(lambda x : dt.date(int(x[0:4]), int(x[5:7]), int(x[8:])).weekday())
    # workday column to check if the day is a part of the weekday or weekend (1 or 0)
    cleaned_df['workday'] = cleaned_df['weekday'].apply(lambda x : 0 if x > 4 else 1)
    # get the total bike user column and remove the trailing zeroes
    cleaned_df['total'] = (cleaned_df['customer'] + cleaned_df['subscriber']).apply(lambda x : int(format(x, '.0f')))
    
    #add this month's dataset onto the array for concatention later
    bike_months.append(cleaned_df)

all_df = pd.concat(bike_months, ignore_index=True)
all_df

,date,hour,customer,subscriber,trip_duration_mean,trip_duration_std,month,season,weekday,workday,total
0,2020-01-01,0,29,48,2841.57,16514.14,1,winter,2,1,77
1,2020-01-01,1,46,65,2692.32,19994.93,1,winter,2,1,111
2,2020-01-01,2,23,13,16587.78,94260.92,1,winter,2,1,36
3,2020-01-01,3,2,7,890.56,420.50,1,winter,2,1,9
4,2020-01-01,4,1,0,114218.00,0.00,1,winter,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...
6544,2020-09-30,19,141,512,1028.88,2281.26,9,fall,2,1,653
6545,2020-09-30,20,89,299,964.61,1051.87,9,fall,2,1,388
6546,2020-09-30,21,70,210,1110.58,2867.71,9,fall,2,1,280
6547,2020-09-30,22,65,168,2218.27,18266.13,9,fall,2,1,233


In [134]:
df.to_csv('bike_dataset-v2.csv')